In [1]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util

In [2]:
import json
import os
import csv
import pickle
import time
import hnswlib

In [3]:
def getNpArr(filepath):
    with open(filepath,'r',encoding='utf-8') as f:
        raw = f.readlines()
    return np.loadtxt(filepath)

In [4]:
arr = getNpArr('D:/起夜急理解/文本挖掘与自然语言理解考核数据文件/题目4/savetest.txt')
arr.shape

(68341, 768)

In [10]:
sentenceTensor = torch.from_numpy(arr)

In [5]:
#提取出所有的问题
def getQuestionList(filepath):
    rawList = []
    with open(filepath,'r',encoding='utf-8') as f:
        raw = f.readlines()
    for item in raw:
        rawList.append(json.loads(item))
    questionList = []
    answerList = []
    for item in rawList:
        questionList.append(item['title'])
        answerList.append(item['content'])
    return questionList, answerList

In [6]:
questionList, answerList = getQuestionList('D:/起夜急理解/文本挖掘与自然语言理解考核数据文件/题目4/webtext2019zh(1)/web_text_zh_testa.txt')

In [26]:
embedding_cache_path = 'D:/起夜急理解/文本挖掘与自然语言理解考核数据文件/题目4/NEWembeddingCache.pkl'
with open(embedding_cache_path, "wb") as fOut:
    pickle.dump({'sentences': questionList, 'answers':answerList, 'embeddings': arr}, fOut)

In [7]:
embedding_cache_path = 'D:/起夜急理解/文本挖掘与自然语言理解考核数据文件/题目4/NEWembeddingCache.pkl'
print("Load pre-computed embeddings from disc")
with open(embedding_cache_path, "rb") as fIn:
    cache_data = pickle.load(fIn)
    questionList = cache_data['sentences']
    answerList = cache_data['answers']
    arr = cache_data['embeddings']

Load pre-computed embeddings from disc


In [8]:
index_path = "J:/SAVEITHERE/hnswlib.index"
index = hnswlib.Index(space = 'ip', dim = 768)
if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
else:
    ### 创建Index
    print("Start creating HNSWLIB index")
    index.init_index(max_elements = len(arr), ef_construction = 400, M = 64)

    # 训练以找到合适的聚类
    index.add_items(arr, list(range(len(arr))))

    print("Saving index to:", index_path)
    index.save_index(index_path)
    print('Saved!')
index.set_ef(50)

Loading index...


In [ ]:
model = SentenceTransformer('stsb-xlm-r-multilingual')
while True:
    inp_question = input("Please enter a question: ")

    start_time = time.time()
    question_embedding = model.encode(inp_question)

    #We use hnswlib knn_query method to find the top_k_hits
    corpus_ids, distances = index.knn_query(question_embedding, k=5)

    # We extract corpus ids and scores for the first query
    hits = [{'corpus_id': id, 'score': 1-score} for id, score in zip(corpus_ids[0], distances[0])]
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    end_time = time.time()

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}\t{}".format(hit['score'], questionList[hit['corpus_id']], answerList[hit['corpus_id']]))

Please enter a question: 考研的意义？
Input question: 考研的意义？
Results (after 8.738 seconds):
	9.999	读研的意义何在？	据我了解，在青岛这样的二线城市，做技术类工作（如研发生产工艺质量也算销售不算了），研究生的工资比本科生的高，涨的也快，因为此类工作对沟通技巧（是技巧，不是能力）要求也不高。考虑问题的思路和解决问题的逻辑也比本科生强一些，自然会更受技术出身的上司喜欢，升迁空间也大。当然，个别本科生（我师父），如果责任心非常强也会逼出比研究生更牛逼的思路和逻辑的。
	9.351	考研应不应该报班？？	同考研 个人觉得还是整一个好 报哪几门看你自己感觉吧 专业课 应该不用了吧 数学英语政治我是打算去搞一搞 相互交流吧 你什么时候考
	9.336	为什么要读研？	小公举中专毕业，工作六年，现在24岁，年入百万，自媒体，目前有两个助理（自媒体甚至不需要员工，单干都可以，主要是在扩大业务线中） 身边研究生，MBA朋友一堆。另外有一次帮意大利某大型集团招人，收到150份硕士以上的简历。虽然我没考大学更没考研，但是对于考研还是尽早工作很有发言权，简单概括一下： 1学历是进入大企业的敲门砖。哪怕我自己是中专，我帮客户也是挑高学历的。不过到了年入百万的咖位，不用投简历了，
	8.934	考研是种什么样的体验？要准备什么？经历什么？	暑假一直在图书馆学习一些新技能，然后准备校招的事情。每天看到很多考研er，然后又想起自己曾经考研的日子，就很想记下一些什么，正好昨天又有学妹找到我问我关于考研的事情，今天就想在这里聊聊“那年考研的那些事儿”，顺便为自己找工作攒攒人品。^^ 嘻嘻 我为什么选择考研？仔细想想，自己最开始决定考研是大二结束之后的暑假。因为高考实在是考的不好，所以本科去了一个不是很心仪的学校，当时想着是想继续深造，
	8.905	保研是什么体验？	11月13日更新 现在距离保研的那段日子已经有段时间了，一直想着来更新我的答案，可是每次想起那段经历总是在喜悦中夹杂着淡淡的遗憾，好奇怪的情绪感觉。所以也一直迟迟没有下笔，不过总是这么拖着也不是办法，总要面对的，再者在保研那段时间，问答社区平台也给了我很多的帮助，我也要回报社会嘛，所以今天终于下定决心来更新了！ 先说一下我本身的情况吧，农学专业。由于受辅